In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

import argparse
import csv
import os
import sys
import re
import argparse
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import torch
import torch.nn as nn
from scipy.stats import loguniform
from scipy.io import loadmat
from sklearn import metrics
from torch.utils import data
from IPython.display import clear_output

#sys.path.append('/content/drive/MyDrive/Tesi_Magistrale')
sys.path.append('D:/Tesi_Magistrale')
from Data_Processing_Utils import Norm_each_sub_by_own_param,NormalizeData, train_val_test_split_df, PlotLoss, plot_confusion_matrix, \
    Get_Sub_Norm_params, windowing_Dataframe

from DATALOADERS import dataframe_dataset_triplet

from MODELS import MKCNN, MKCNN_grid, random_choice, train_model_triplet, pre_train_model_triplet, train_model_standard, MultiKernelConv2D_grid


In [18]:
# custom pandas dataset class modified to provide also the angle
class Pandas_Dataset_angle(data.Dataset):

    def __init__(self, df_grouped_by_samples, return_sub=False):
        self.grouped = df_grouped_by_samples
        self.channels = [i for i in df_grouped_by_samples.obj.columns if 'ch' in i]
        self.indices = list(df_grouped_by_samples.indices)
        self.return_sub = return_sub

    def __len__(self):
        return len(self.grouped)

    def __getitem__(self, index):
        picked_smp = self.grouped.get_group(self.indices[index])
        # Picking only the channels columns from a single sample
        sample = torch.tensor(picked_smp.loc[:, self.channels].values).type(torch.float32)
        # picking only one label for each sample
        label = torch.tensor(picked_smp.loc[:, ['label']].head(1).values[0][0]).type(torch.int8)
        angle = torch.tensor(picked_smp.loc[:, ['angle']].head(1).values[0][0]).type(torch.int16)
        if self.return_sub:
            # picking only one subject
            sub = torch.tensor(picked_smp.loc[:, ['sub']].head(1).values[0][0]).type(torch.int8)
            return sample, label, angle, sub
        else:

            # It's missing the part in which I Normalize the data for each subject, or for each subject and channels
            # It's not a good idea to do that on the dataloader while producing results
            return sample, label, angle


In [25]:
#file_path = '/content/drive/MyDrive/Tesi_Magistrale'
#data_path = '/content/drive/MyDrive/Tesi_Magistrale/DATA/'

file_path = 'D:/Tesi_Magistrale'
data_path = 'D:/Tesi_Magistrale/DATA/'

subjects = ['Sub1', 'Sub2', 'Sub3']
#subj = 'Manfredo'
day = 1
single_model = False
test_rep = 3

# constants
WND_LEN = 200
NORM_MODE = 'sub'
NORM_TYPE = 'zero_to_one'
RECT = True
split_mode = 'rep'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

poses = ['medium_wrap', 'lateral', 'power_sphere', 'power_disk', 'prismatic_pinch', 'index_extension',
                    'wave_out', 'wave_in', 'fist', 'open_hand']


Using device: cuda


In [28]:
for subj in subjects:
    for prova in range (1,4):
        print('\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
        save_path = data_path + f"{subj}_offset_relabel/day{day}/Prova_{prova}/Equal_Dataframe_Test/Cross_rep/single_angle/Cross_rep/Angle_Test/"
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
        # Preparing Dataframe
        df_path = data_path + f"{subj}_offset_relabel/day{day}/Dataframe/dataframe_wnd_{WND_LEN}_angle_merged.csv"
        df = pd.read_csv(df_path)
        # Using only test rep 3 to be fair, since the single model is tested only on that
        df = Norm_each_sub_by_own_param(df, mode= 'sub', norm_type= 'zero_to_one', rectify=True )
        df = df[df['rep'] == test_rep]
       
            
        print(f"Loaded dataframe of subject {subj} for test rep {test_rep}")
        weight_path = data_path + f"{subj}_offset_relabel/day{day}/Prova_{prova}/Equal_Dataframe_Test/Cross_rep/single_angle/Cross_rep/Best_States/"
        weight_list = os.listdir(weight_path)
        
        for weight in weight_list: # for each model stored
            print("\n------------------------------------------------------------------------------")
            
            
            single_model = 'single_model' in weight
            if single_model:
                current_model = 'single_model'
            elif 'ang_1' in weight:
                current_model = 'angle_1'
            elif 'ang_2' in weight:
                current_model = 'angle_2'
            elif 'ang_3' in weight:
                current_model = 'angle_3'
            elif 'ang_4' in weight:
                current_model = 'angle_4'
            else:
                current_model = 'angle_5'
            
                
            print(f"\nLoaded {weight}")
            print(f"Testing Single Model {single_model}")
            # Preparing Model
            kernels_gap = [g for g in range(0, 3 * round(WND_LEN / 20), round(WND_LEN / 20))]
            kernel_sizes = np.full((3, 5, 2), [1, 3])
            for j in range(3):
                for i in range(5):
                    kernel_sizes[j][i][0] = (round(WND_LEN / 20) * (i + 1) + kernels_gap[j])
    
            n_classes = 10
    
            net = {'N_multik': 32, 'N_Conv_conc': 64, 'N_SepConv': 128,
            'Kernel_multi_dim': kernel_sizes[0],
            'Kernel_Conv_conc': 1,
            'act_func': nn.ReLU(),
            'Pool_Type': nn.MaxPool2d,
            'wnd_len':WND_LEN
            }
    
            model = MKCNN_grid(net, num_classes=n_classes)
            model = model.to(device)
            softmax_block = nn.Softmax(dim=1)
            print("Model Built")
            w = torch.load(weight_path+weight, map_location = torch.device(device))
            model.load_state_dict(w)
            print("Model Ready")
    
            for ANGLE in range(1,6): # for each angle
                print(f"Testing {current_model} on angle {ANGLE}")    
                # loading angle dataframe
                current_df = df[df['angle_index']==ANGLE]
                current_df = Pandas_Dataset_angle(current_df.groupby('sample_index'))
                batch_size = 1
                num_workers = 0
                params = {'batch_size': batch_size,
                            'shuffle': False,
                            # 'sampler': sampler,
                            'num_workers': num_workers,
                            'drop_last': False}
                current_df_dataloader = data.DataLoader(current_df, **params)
    
                model.eval()
                with torch.no_grad():
                    y_pred = []
                    y_true = []
                    for inputs, labels, angle in current_df_dataloader:
                        inputs = inputs[:, None, :, :]
                        inputs = inputs.to(device)
                        angle = abs(angle.cpu().data.numpy()[0])
                        labels_np = labels.cpu().data.numpy()
                        
                        output, _ = model.forward(inputs)
                        outputs_np = softmax_block(output)
                        
                        outputs_np = outputs_np.cpu().data.numpy()
                        outputs_np = np.argmax(outputs_np, axis=1)
                        y_pred = np.append(y_pred, outputs_np)
                        y_true = np.append(y_true, labels_np)
                        print(f"Model {current_model}\tAngle: {ANGLE}\tPredicted pose: {outputs_np}\tReal Pose: {labels_np}", end = '\r')
                print(f"\nTesting for angle {ANGLE} Done!")
                cm = metrics.confusion_matrix(y_true, y_pred)
                title = f"day{day} {'single model' if single_model else 'multi model'} rep {test_rep}" # title on the confusion matrix
                if not os.path.exists(save_path + f'{current_model}/'):
                    os.makedirs(save_path + f'{current_model}/')
                plot_confusion_matrix(cm, target_names=poses, title=f'Confusion Matrix {current_model.replace("_"," ")} on angle {ANGLE}',path_to_save=save_path + f'{current_model}/cm_day{day}_zero_to_one_{current_model}_on_angle_{ANGLE}.png')
        print(f"Testing for sub {subj} Done!")
        clear_output()
    
    
                
    
    
    
            